## New data structure

#### Analysts only in 2 levels

#### Capacity Scale 6-14 

#### Come in sub departments

#### Max task value 1.5


In [45]:
from collections import deque

class Analyst:
    def __init__(self, name, rating, team, workload=0, max_workload=40, available=True):
        self.name = name
        self.rating = rating  #1 for Standard payroll analyst 2 for Senior
        self.workload = workload #Current workload weight 
        self.max_workload = max_workload # Max Capacity
        self.available = available # True if available for new tasks
        self.absent = False
        self.tasks = [] # List of tasks assigned to the analyst
        self.team = team # team A or B but can be customised

class Task:
    def __init__(self, name, importance, effort, team):
        self.name = name
        self.importance = importance  # label 1 if can be completed by any employee two if only seniors can complete it
        self.effort = effort # How much workload required to complete task
        self.assigned_to = None
        self.team = team # team A or B but can be customised

In [50]:
def find_eligible_analyst(task, candidates): ##helper function
    """
    Returns the best available analyst from the candidate list who meets the rating 
    and workload requirements for the given task.
    """
    for analyst in sorted(candidates, key=lambda a: (-a.rating, a.workload)):
        if analyst.available and (analyst.workload + task.effort <= analyst.max_workload):
            if analyst.rating >= task.importance:
                return analyst
    return None


In [ ]:
def assign_tasks_max_level(analysts_dict: dict, tasks: list): 
    """Analysts are assigned tasks based on their rating, workload, and team.
    Tasks are sorted by importance and effort, with higher importance tasks prioritized.
    If no suitable analyst is found, the task is added to a list of unassigned tasks.
    """
    analysts = list(analysts_dict.values())
    task_queue = deque(sorted(tasks, key=lambda t: (-t.importance, t.effort)))
    unassigned_tasks = []

    while task_queue:
        task = task_queue.popleft()
        team_analysts = [a for a in analysts if a.team == task.team]
        all_analysts_sorted = sorted(analysts, key=lambda a: (-a.rating, a.workload))


        best_analyst = find_eligible_analyst(task, team_analysts)

        if not best_analyst:
            print(f"No available analyst in {task.team} for {task.name}. Opening to department.")
            best_analyst = find_eligible_analyst(task, all_analysts_sorted)

        if best_analyst:
            best_analyst.tasks.append(task)
            best_analyst.workload += task.effort
            best_analyst.available = best_analyst.workload < best_analyst.max_workload
            task.assigned_to = best_analyst.name
            print(f"Assigned {task.name} to {best_analyst.name}")
        else:
            hours_remaining = sorted(analysts, key=lambda a: - (a.max_workload - a.workload))
            if task.effort > (hours_remaining[0].max_workload - hours_remaining[0].workload):
                print(
                    f"""No available analyst for {task.name}. The task requires {task.effort} hours, but the employee with 
the most available hours is {hours_remaining[0].name}, who has {hours_remaining[0].max_workload - hours_remaining[0].workload} hours available.
Consider a temporary contractor."""
                )
            else:
                print(f"No analyst found for task {task.name}.")
            unassigned_tasks.append(task)

    if unassigned_tasks:
        print(f"WARNING: Tasks unable to be assigned are {[task.name for task in unassigned_tasks]}..Attempting reassignment")
        sorted_reassignment = deque(sorted(unassigned_tasks, key=lambda t: (-t.importance)))
        potential_analysts=[]
        for analyst in analysts:
            if analyst.available and (analyst.rating >= sorted_reassignment[0].importance):
                potential_analysts.append(analyst)
        print('Potential analysts:',[analyst.name for analyst in potential_analysts])
        for analyst in potential_analysts:            
            for task in analyst.tasks:              
                if task.importance < sorted_reassignment[0].importance:
                    print('Attempting recursive reassignment, currently reasigning tasks for', analyst.name)
                    reassign_tasks_reprioritise(analysts_dict,analyst.name,sorted_reassignment[0])   
                    break
                else:
                    print(f"All tasks for {analyst.name} are at or above the importance level of {sorted_reassignment[0].importance}. No reassignment reccomended.")
                    break
                    


In [ ]:
def reassign_tasks_reprioritise(analysts_dict:dict ,analyst:str,task:object):
    """Analysts are reassigned tasks based on their current workload and the importance of the task that needs reassignmend.
    If the analyst has tasks that are below the importance of the task being reassigned,
    those tasks are removed and the new task is added to their workload.
    If no tasks are below the importance level, the analyst's workload unchanged.
    """
    tasks_for_reassignment=analysts_dict[analyst].tasks
    analysts_dict[analyst].workload=0
    analysts_dict[analyst].tasks = [] # List of tasks assigned to the analyst
    tasks_for_reassignment.append(task)
    print('tasks for reassignment:',[task.name for task in tasks_for_reassignment])
    analysts = list(analysts_dict.values())
    task_queue = deque(sorted(tasks_for_reassignment, key=lambda t: (-t.importance, t.effort)))
    unnasigned_tasks=[]
    
    while task_queue:
        task = task_queue.popleft()
        team_analysts = [a for a in analysts if a.team == task.team]
        all_analysts_sorted = sorted(analysts, key=lambda a: (-a.rating, a.workload))


        best_analyst = find_eligible_analyst(task, team_analysts)

        if not best_analyst:
            print(f"No available analyst in {task.team} for {task.name}. Opening to department.")
            best_analyst = find_eligible_analyst(task, all_analysts_sorted)

        if best_analyst:
            best_analyst.tasks.append(task)
            best_analyst.workload += task.effort
            best_analyst.available = best_analyst.workload < best_analyst.max_workload
            task.assigned_to = best_analyst.name
            print(f"Assigned {task.name} to {best_analyst.name}")
        else:
            hours_remaining = sorted(analysts, key=lambda a: - (a.max_workload - a.workload))
            if task.effort > (hours_remaining[0].max_workload - hours_remaining[0].workload):
                print(
                    f"""No available analyst for {task.name}. The task requires {task.effort} hours, but the employee with 
the most available hours is {hours_remaining[0].name}, who has {hours_remaining[0].max_workload - hours_remaining[0].workload} hours available.
Consider a temporary contractor."""
                )
            else:
                print(f"No analyst found for task {task.name}.")
            unnasigned_tasks.append(task)

    if unnasigned_tasks:
        print(f"WARNING: Tasks unable to be assigned are {[task.name for task in unnasigned_tasks]}..Attempting reassignment")
        sorted_reassignment = deque(sorted(unnasigned_tasks, key=lambda t: (-t.importance)))
        potential_analysts=[]
        for analyst in analysts:
            if analyst.available and (analyst.rating >= sorted_reassignment[0].importance):
                potential_analysts.append(analyst)
        print('Potential analysts:',[analyst.name for analyst in potential_analysts])
        for analyst in potential_analysts:            
            for task in analyst.tasks:              
                if task.importance < sorted_reassignment[0].importance:
                    print('Attempting recursive reassignment, currently reasigning tasks for', analyst.name)
                    reassign_tasks_reprioritise(analysts_dict,analyst.name,sorted_reassignment[0])   
                    break
                else:
                    print(f"All tasks for {analyst.name} are at or above the importance level of {sorted_reassignment[0].importance}. No reassignment reccomended.")
                    break
        
        
    

In [ ]:
def reassign_tasks_unavailability(analysts_dict:dict ,unavailable_analyst:str):
    """Analysts are reassigned tasks when an analyst becomes unavailable.
    The unavailable analyst's tasks are reassigned to other analysts based on their availability and workload.
    If no suitable analyst is found, the task is added to a list of unassigned tasks.
    """
    print('Unavailable analyst:', analysts_dict[unavailable_analyst].name)
    analysts_dict[unavailable_analyst].available = False
    tasks_for_reassignment=analysts_dict[unavailable_analyst].tasks
    print('Tasks for reassignment:',[task.name for task in tasks_for_reassignment])
    analysts = list(analysts_dict.values())
    task_queue = deque(sorted(tasks_for_reassignment, key=lambda t: (-t.importance, t.effort)))
    unnasigned_tasks=[]
    
    while task_queue:
        task = task_queue.popleft()
        team_analysts = [a for a in analysts if a.team == task.team]
        all_analysts_sorted = sorted(analysts, key=lambda a: (-a.rating, a.workload))


        best_analyst = find_eligible_analyst(task, team_analysts)

        if not best_analyst:
            print(f"No available analyst in {task.team} for {task.name}. Opening to department.")
            best_analyst = find_eligible_analyst(task, all_analysts_sorted)

        if best_analyst:
            best_analyst.tasks.append(task)
            best_analyst.workload += task.effort
            best_analyst.available = best_analyst.workload < best_analyst.max_workload
            task.assigned_to = best_analyst.name
            print(f"Assigned {task.name} to {best_analyst.name}")
        else:
            hours_remaining = sorted(analysts, key=lambda a: - (a.max_workload - a.workload))
            if task.effort > (hours_remaining[0].max_workload - hours_remaining[0].workload):
                print(
                    f"""No available analyst for {task.name}. The task requires {task.effort} hours, but the employee with 
the most available hours is {hours_remaining[0].name}, who has {hours_remaining[0].max_workload - hours_remaining[0].workload} hours available.
Consider a temporary contractor."""
                )
            else:
                print(f"No analyst found for task {task.name}.")
            unnasigned_tasks.append(task)

    if unnasigned_tasks:
        print(f"WARNING: Tasks unable to be assigned are {[task.name for task in unnasigned_tasks]}..Attempting reassignment")
        sorted_reassignment = deque(sorted(unnasigned_tasks, key=lambda t: (-t.importance)))
        potential_analysts=[]
        for analyst in analysts:
            if analyst.available and (analyst.rating >= sorted_reassignment[0].importance):
                potential_analysts.append(analyst)
        print('Potential analysts:',[analyst.name for analyst in potential_analysts])
        for analyst in potential_analysts:            
            for task in analyst.tasks:              
                if task.importance < sorted_reassignment[0].importance:
                    print('Attempting recursive reassignment, currently reasigning tasks for', analyst.name)
                    reassign_tasks_reprioritise(analysts_dict,analyst.name,sorted_reassignment[0])   
                    break
                else:
                    print(f"All tasks for {analyst.name} are at or above the importance level of {sorted_reassignment[0].importance}. No reassibgnment reccomended.")
                    break
                    
        
        

#### Example usgage - max capacity set to 6 to save generating 100s of tasks

In [60]:
analysts_dict = {
    "Alice": Analyst("Alice", 2, 'TeamA', 0, 6), 
    "Bob": Analyst("Bob", 2, 'TeamA', 0, 6),
    "Dana": Analyst("Dana", 2, 'TeamB', 0, 6),
    "Eli": Analyst("Eli", 1, 'TeamA', 0, 6),
    "Freddie":Analyst("Freddie", 1, 'TeamB', 0, 6),
    "Graham":Analyst("Graham",1, 'TeamB', 0, 6) 
}

tasks = [
    Task("Task1", 2, 1.5, 'TeamA'),
    Task("Task2", 1, 0.3, 'TeamA'),
    Task("Task3", 1, 0.3, 'TeamA'),
    Task("Task4", 2, 1.0, 'TeamB'),
    Task("Task5", 1, 0.5, 'TeamB'),
    Task("Task6", 1, 0.7, 'TeamB'),
    Task("Task7", 2, 1.0, 'TeamB'),
    Task("Task8", 1, 0.4, 'TeamB'),
    Task("Task9", 2, 1.2, 'TeamA'),
    Task("Task10", 1, 0.6, 'TeamA'),
    Task("Task11", 2, 1.3, 'TeamB'),
    Task("Task12", 1, 0.8, 'TeamB'),
    Task("Task13", 2, 1.4, 'TeamA'),
    Task("Task14", 1, 0.9, 'TeamA'),
    Task("Task15", 2, 1.1, 'TeamB'),
    Task("Task16", 1, 0.2, 'TeamB'),
]
assign_tasks_max_level(analysts_dict,tasks)
reassign_tasks_unavailability(analysts_dict,"Alice")


Assigned Task4 to Dana
Assigned Task7 to Dana
Assigned Task15 to Dana
Assigned Task9 to Alice
Assigned Task11 to Dana
Assigned Task13 to Bob
Assigned Task1 to Alice
Assigned Task16 to Dana
Assigned Task2 to Bob
Assigned Task3 to Bob
Assigned Task8 to Dana
Assigned Task5 to Dana
Assigned Task10 to Bob
Assigned Task6 to Freddie
Assigned Task12 to Graham
Assigned Task14 to Bob
Unavailable analyst: Alice
Tasks for reassignment: ['Task9', 'Task1']
Assigned Task9 to Bob
No available analyst in TeamA for Task1. Opening to department.
No analyst found for task Task1.
Potential analysts: ['Bob', 'Dana']
All tasks for Bob are at or above the importance level of 2. No reassignment reccomended.
Attempting recursive reassignment, currently reasigning tasks for Bob
tasks for reassignment: ['Task13', 'Task2', 'Task3', 'Task10', 'Task14', 'Task9', 'Task1']
Assigned Task9 to Bob
Assigned Task13 to Bob
Assigned Task1 to Bob
Assigned Task2 to Bob
Assigned Task3 to Bob
Assigned Task10 to Bob
Assigned Task